#Graphs & Social Networks &mdash; lab material
This notebook contains a definition of a python class *Graph*, containing functions described during labs.

Package import.

In [1]:
import numpy as np
from random import random, seed
from copy import deepcopy
from collections import deque
import time

## Simple functions from the first classes
This is left just as an example (to compare the behaviour).

In [2]:
def print_matrix(vertices, matrix):
  """
  Printing a graph given by adjacency matrix
  """
  n = len(matrix)
  if (vertices is not None) and (len(vertices) == n):
    vv = vertices
  else:
    vv = range(1, n+1)
  for i in range(n):
    print(vv[i], ":", end="")
    for j in range(n):
      if matrix[i, j]:
        print(" ", vv[j], end="")
    print("")

def print_dict(graph):
  """
  Printing of a graph (given as a dictionary/neighbouring list)
  """
  for v in graph:
    print(v, ":", end="")
    for u in graph[v]:
      print(" ", u, end="")
    print("")

## Class *Graph*

In [64]:
class Graph:
    def __init__(self, graph=None, weights=None):
        if graph is None:
            graph = {}
        self.graph = graph
        # Dictionary to map edge (u, v) -> weight
        # For undirected edges {u, v}, we store both (u, v) and (v, u)
        self.weights = weights if weights is not None else {}

    # dict initializer
    @classmethod
    def from_dict(cls, graph):
        return cls(graph)

    # array initializer
    @classmethod
    def from_matrix(cls, matrix, vertices=None):
        if (vertices is None) or (len(vertices) != len(matrix)):
            vertices = [*range(1, len(matrix) + 1)]
        return cls.from_dict(cls._matrix_to_dict(matrix, vertices))

    # weighted matrix initializer (Task 2)
    @classmethod
    def from_weighted_matrix(cls, matrix, vertices=None):
        """
        Creates a graph from a weighted matrix (Task 2).
        matrix[i][j] should be the weight. Use np.inf or 0 for no edge.
        """
        if (vertices is None) or (len(vertices) != len(matrix)):
            vertices = [*range(1, len(matrix) + 1)]

        g = cls()
        n = len(matrix)

        # Add all vertices first
        for v in vertices:
            g.add_vertex(v)

        for i in range(n):
            for j in range(n):
                weight = matrix[i][j]
                # Assuming non-edges are represented by infinity or 0 (if 0 is not a valid weight)
                # For distance matrices, 0 usually means self-loop or free edge, so we check for Inf usually.
                # We'll assume if weight < np.inf and weight != 0 (unless i==j) it's an edge.
                # Or explicitly check if it's not "no connection".
                if weight != np.inf and i != j:
                    # We assume directed for general matrix, unless symmetric check is done.
                    # Using add_weighted_arc for generality.
                    g.add_weighted_arc((vertices[i], vertices[j]), weight)
        return g

    # two private methods matrix <-> dictionaries
    @staticmethod
    def _matrix_to_dict(matrix, vertices: list) -> dict:
        """
        Converts a graph given as an adjacency matrix to a graph in dict form.
        """
        res_dict = {}
        for i, v in enumerate(vertices):
            # treating > 0 or True as edge
            neighbours = [vertices[j] for j, edge in enumerate(matrix[i]) if edge]
            res_dict[v] = neighbours
        return res_dict

    def _dict_to_matrix(self, _dict: dict) -> np.array:
        """
        Converts a graph in dict form to its adjacency matrix (unweighted).
        """
        n = len(_dict)
        vertices = [*_dict.keys()]
        matrix = np.zeros(shape=(n, n), dtype=int)
        for u, row in _dict.items():
            for v in row:
                if u in vertices and v in vertices:
                    u_idx = vertices.index(u)
                    v_idx = vertices.index(v)
                    matrix[u_idx][v_idx] += 1
        return matrix

    def vertices(self) -> list:
        """
        Returns list of vertices of the graph.
        """
        return [*self.graph.keys()]

    def matrix(self) -> np.array:
        """
        Returns the adjacency matrix of the graph.
        """
        return self._dict_to_matrix(self.graph)

    # redefinition of print for objects of class Graph
    def __str__(self):
        res = ""
        for v in self.graph:
            res += f"{v}:"
            for u in self.graph[v]:
                res += f" {u}"
                if (v, u) in self.weights:
                     res += f"({self.weights[(v, u)]})"
            res += "\n"
        return res

    def to_neighbourlist(self, filename: str):
        """
        Saves a graphs to a text file as a neighbour dict.
        Filename is a file path.
        """
        file = open(filename, "w")
        file.write(str(self))
        file.close()

    # Modifying graphs
    def add_vertex(self, vertex):
        """
        Adds a new vertex to the graph.
        """
        if vertex not in self.graph:
            self.graph[vertex] = []

    def del_vertex(self, vertex):
        """
        Removes a vertex from the graph.
        """
        if vertex in self.graph:
            self.graph.pop(vertex)
            # Remove edges pointing to vertex
            for u in self.graph:
                if vertex in self.graph[u]:
                    self.graph[u].remove(vertex)
            # Cleanup weights
            keys_to_remove = [k for k in self.weights if vertex in k]
            for k in keys_to_remove:
                del self.weights[k]

    def add_arc(self, arc):
        """
        Given pair of vertices (arc variable) add an arc to the graph.
        """
        u, v = arc
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        # Default weight 1 if not exists
        if (u, v) not in self.weights:
            self.weights[(u, v)] = 1

    def add_weighted_arc(self, arc, weight):
        """
        Adds a weighted directed edge.
        """
        u, v = arc
        self.add_arc(arc)
        self.weights[(u, v)] = weight

    def add_edge(self, edge: list):
        """
        Given pair of vertices (edge variable) add an edge to existing graph.
        """
        u, v = edge
        if u == v:
            raise ValueError("Loops are not allowed!")
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.graph[u]:
            self.graph[u].append(v)
        if u not in self.graph[v]:
            self.graph[v].append(u)
        # Default weight 1
        if (u, v) not in self.weights:
            self.weights[(u, v)] = 1
            self.weights[(v, u)] = 1

    def add_weighted_edge(self, edge, weight):
        """
        Adds a weighted undirected edge.
        """
        u, v = edge
        self.add_edge(edge)
        self.weights[(u, v)] = weight
        self.weights[(v, u)] = weight

    # Reading from a file
    @staticmethod
    def from_edges(filename: str, directed=0):
        """
        Read the graph from file.
        """
        graph = Graph()
        try:
            file = open(filename, "r")
            for line in file:
                words = line.strip().split()
                if len(words) == 1:
                    graph.add_vertex(words[0])
                elif len(words) >= 2:
                    if directed:
                        graph.add_arc([words[0], words[1]])
                    else:
                        graph.add_edge([words[0], words[1]])
                    # Note: This basic parser does not handle weights in file
            file.close()
        except FileNotFoundError:
            print(f"File {filename} not found.")
        return graph

    @staticmethod
    def random_graph(n: int, p: float):
        """
        Creates a random graph in G(n, p) model.
        """
        rand_graph = Graph()
        for i in range(1, n + 1):
            rand_graph.add_vertex(i)
            for j in range(1, i):
                if random() < p:
                    rand_graph.add_edge([i, j])
        return rand_graph

    @staticmethod
    def cycle(n: int):
        """
        Creates a cycle C_n on n vertices
        """
        cycle = Graph()
        for i in range(n - 1):
            cycle.add_edge([i + 1, i + 2])
        cycle.add_edge([1, n])
        return cycle

    def Prufer(self):
        """
        Returns the Prufer code of a tree.
        """
        tr = deepcopy(self.graph)
        code = ""
        for i in range(len(self.graph) - 2):
            leaf = None
            for x in sorted(tr):
                if len(tr[x]) == 1:
                    leaf = x
                    break

            if leaf is None:
                break

            v = tr[leaf][0]
            code = code + f"{v} "
            tr[v].remove(leaf)
            tr.pop(leaf)
        return code.strip()

    @staticmethod
    def tree_from_Prufer(code: str):
        """
        Creating a tree from a Prufer code.
        """
        tree = Graph()
        clist = [int(x) for x in code.strip().split()]
        n = len(clist) + 2
        vert = [*range(1, n + 1)]
        for v in vert:
            tree.add_vertex(v)
        for i in range(n - 2):
            for x in vert:
                if x not in clist:
                    break
            v = clist.pop(0)
            tree.add_edge((x, v))
            vert.remove(x)
        if len(vert) >= 2:
            tree.add_edge(vert)
        return tree

    @staticmethod
    def random_bipartite_graph(m: int, n: int, p: float):
        """
        Generate an undirected bipartite random graph
        """
        if m < 0 or n < 0:
            raise ValueError("m and n must be non-negative.")
        if not (0.0 <= p <= 1.0):
            raise ValueError("p must be in [0, 1].")

        G = Graph()
        for v in range(1, m + n + 1):
            G.add_vertex(v)

        for u in range(1, m + 1):
            for w in range(m + 1, m + n + 1):
                if random() < p:
                    G.add_edge([u, w])

        return G

    def connected_components(self):
        """
        Looks for connected components of undirected graph via DFS.
        """
        def DFS(u):
            for w in self.graph[u]:
                if w not in VT[0]:
                    VT[0].add(w)
                    VT[-1].add(w)
                    DFS(w)

        VT = [set([])]
        for v in self.graph:
            if v not in VT[0]:
                VT[0].add(v)
                VT.append(set([v]))
                DFS(v)
        return VT

    def preorder(self, v):
        """
        Print vertices of a tree in preorder, starting from vertex v.
        """
        visited = set()
        order = []

        def DFS(u, parent=None):
            visited.add(u)
            order.append(u)
            for w in sorted(self.graph.get(u, [])):
                if w not in visited and w != parent:
                    DFS(w, u)

        DFS(v)
        return " ".join(map(str, order))


    def postorder(self, v):
        """
        Print vertices of a tree in postorder, starting from vertex v.
        """
        visited = set()
        order = []

        def DFS(u, parent=None):
            visited.add(u)
            for w in sorted(self.graph.get(u, [])):
                if w not in visited and w != parent:
                    DFS(w, u)
            order.append(u)

        DFS(v)
        return " ".join(map(str, order))


    def ConnectedComponentsGraphs(self):
        """
        Returns a list of Graph objects, each corresponding to one connected component.
        """
        VT = self.connected_components()
        components = []

        for i, vertex_set in enumerate(VT[1:], start=1):
            sub_dict = {
                v: [w for w in self.graph[v] if w in vertex_set]
                for v in vertex_set
            }
            subgraph = Graph.from_dict(sub_dict)
            components.append(subgraph)

            print(f"Connected Component {i}:")
            print(subgraph)

        return components

    def connected_components_bfs(self):
        """
        Iterative BFS for connected components.
        """
        VT = [set([])]
        for v in self.graph:
            if v not in VT[0]:
                component = set([v])
                VT[0].add(v)
                queue = deque([v])
                while queue:
                    u = queue.popleft()
                    for w in self.graph[u]:
                        if w not in VT[0]:
                            VT[0].add(w)
                            component.add(w)
                            queue.append(w)
                VT.append(component)
        return VT

    def topological_sort(self):
        """
        Kahn's algorithm for topological sorting.
        """
        in_degree = {v: 0 for v in self.graph}
        for u in self.graph:
            for v in self.graph[u]:
                if v not in in_degree: in_degree[v] = 0
                in_degree[v] += 1

        queue = deque([v for v in in_degree if in_degree[v] == 0])
        topo_order = []

        while queue:
            u = queue.popleft()
            topo_order.append(u)
            for v in self.graph[u]:
                in_degree[v] -= 1
                if in_degree[v] == 0:
                    queue.append(v)

        if len(topo_order) != len(in_degree):
            raise ValueError("Graph has a cycle! Topological sort not possible.")

        return topo_order

    # -------------------------------------------------------------------------
    # TASK 1: Strongly Connected Components (Kosaraju's Algorithm)
    # -------------------------------------------------------------------------
    def get_transpose(self):
        """
        Returns the transpose of the graph (edges reversed).
        """
        g_t = Graph()
        for v in self.graph:
            g_t.add_vertex(v)
        for u in self.graph:
            for v in self.graph[u]:
                # Arc u -> v becomes v -> u
                # Preserve weights if they exist
                w = self.weights.get((u, v), 1)
                g_t.add_weighted_arc((v, u), w)
        return g_t

    def strongly_connected_components(self):
        """
        Returns a list of strongly connected components using Kosaraju's algorithm.
        Hint: Uses stack based on finishing times (related to topological sorting).
        """
        stack = []
        visited = set()

        # Step 1: Fill vertices in stack according to finishing times
        def fill_order(u):
            visited.add(u)
            for v in self.graph[u]:
                if v not in visited:
                    fill_order(v)
            stack.append(u)

        for v in self.graph:
            if v not in visited:
                fill_order(v)

        # Step 2: Get Transpose Graph
        g_t = self.get_transpose()

        # Step 3: Process all vertices in order defined by stack
        visited.clear()
        sccs = []

        def dfs_util(u, current_component):
            visited.add(u)
            current_component.append(u)
            for v in g_t.graph[u]:
                if v not in visited:
                    dfs_util(v, current_component)

        while stack:
            u = stack.pop()
            if u not in visited:
                component = []
                dfs_util(u, component)
                sccs.append(component)

        return sccs

    # -------------------------------------------------------------------------
    # TASK 2: Weighted Matrix Conversions
    # -------------------------------------------------------------------------
    def to_weighted_matrix(self):
        """
        Converts graph to a weighted adjacency matrix.
        Non-edges are represented by np.inf.
        Returns: matrix (np.array), vertices_map (dict)
        """
        vertices = self.vertices()
        n = len(vertices)
        v_map = {v: i for i, v in enumerate(vertices)}

        matrix = np.full((n, n), np.inf)
        np.fill_diagonal(matrix, 0)

        for u in self.graph:
            for v in self.graph[u]:
                u_idx = v_map[u]
                v_idx = v_map[v]
                # Use stored weight or default to 1
                w = self.weights.get((u, v), 1)
                matrix[u_idx][v_idx] = w

        return matrix, v_map

    # -------------------------------------------------------------------------
    # TASK 3: Floyd-Warshall (Weighted)
    # -------------------------------------------------------------------------
    def floyd_warshall(self):
        """
        Computes all-pairs shortest paths using Floyd-Warshall.
        Uses the weighted matrix from to_weighted_matrix().
        """
        dist, v_map = self.to_weighted_matrix()
        n = len(dist)

        # The algorithm
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    # If path through k is shorter, update
                    if dist[i][j] > dist[i][k] + dist[k][j]:
                        dist[i][j] = dist[i][k] + dist[k][j]

        return dist, v_map

## Use of code

### Part 4 (Connected components via DFS)

In [65]:
graph1 = Graph.random_graph(10, 1/3)
print(graph1)

1: 4(1) 6(1) 10(1)
2: 4(1) 9(1)
3: 7(1) 9(1)
4: 1(1) 2(1) 8(1) 9(1)
5: 8(1) 9(1) 10(1)
6: 1(1) 8(1) 9(1)
7: 3(1)
8: 4(1) 5(1) 6(1) 9(1)
9: 2(1) 3(1) 4(1) 5(1) 6(1) 8(1)
10: 1(1) 5(1)



In [66]:
print(len(graph1.connected_components()))
print(graph1.connected_components())

2
[{1, 2, 3, 4, 5, 6, 7, 8, 9, 10}, {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}]


### Part 3 (Prufer codes)

In [67]:
%%writefile tree1.txt
1 3
3 5
2 5
4 5
6 5

Overwriting tree1.txt


In [68]:
tree1 = Graph.from_edges("tree1.txt")

In [69]:
print(tree1)

1: 3(1)
3: 1(1) 5(1)
5: 3(1) 2(1) 4(1) 6(1)
2: 5(1)
4: 5(1)
6: 5(1)



In [70]:
print(tree1.Prufer())

3 5 5 5


In [71]:
print(Graph.tree_from_Prufer("3 5 5 5"))

1: 3(1)
2: 5(1)
3: 1(1) 5(1)
4: 5(1)
5: 2(1) 3(1) 4(1) 6(1)
6: 5(1)



In [72]:
print(Graph.tree_from_Prufer("5 4 3 1"))

1: 3(1) 6(1)
2: 5(1)
3: 4(1) 1(1)
4: 5(1) 3(1)
5: 2(1) 4(1)
6: 1(1)



In [73]:
print(Graph.tree_from_Prufer("1 1 1 2 3"))

1: 4(1) 5(1) 6(1) 2(1)
2: 1(1) 3(1)
3: 2(1) 7(1)
4: 1(1)
5: 1(1)
6: 1(1)
7: 3(1)



## Use of homework

**1)** Write a preorder(v) and postorder(v) function that will print trees (e.g. generated from Prüfer's code) in preorder or (respectively) postorder order, starting from vertex v.

In [74]:
tree = Graph.tree_from_Prufer("3 5 5 5")

In [75]:
tree.preorder(5)

'5 2 3 1 4 6'

In [76]:
tree.postorder(5)

'2 1 3 4 6 5'

**2)** The ConnectedComponents function shown during classes returns a list of vertex sets. Write a ConnectedComponentsGraphs() function that returns a list of graphs — connected components of the graph on which is run. One can (worthwhile) use the (ready) ConnectedComponents function.

In [77]:
graph1.ConnectedComponentsGraphs();

Connected Component 1:
1: 4 6 10
2: 4 9
3: 7 9
4: 1 2 8 9
5: 8 9 10
6: 1 8 9
7: 3
8: 4 5 6 9
9: 2 3 4 5 6 8
10: 1 5



**3)** Write a random_bipartite_graph(m,n, p)  function that will generate a bipartite random graph with m+n vertices (a subgraph of the graph K(m,n ) in which each possible pair of vertices is connected by an edge independently, with probability p.

In [78]:
B = Graph.random_bipartite_graph(4, 4, 0.4)
print(B)

1: 5(1)
2: 8(1)
3: 5(1) 6(1)
4: 5(1) 8(1)
5: 1(1) 3(1) 4(1)
6: 3(1)
7:
8: 2(1) 4(1)



### Part2

In [79]:
%%writefile edges.txt
a b
b c
b d
d
d c
e
f


Overwriting edges.txt


In [80]:
%cat edges.txt

a b
b c
b d
d
d c
e
f


In [81]:
graph2 = Graph.from_edges("edges.txt")
print(graph2)

a: b(1)
b: a(1) c(1) d(1)
c: b(1) d(1)
d: b(1) c(1)
e:
f:



In [82]:
graph2.to_neighbourlist("neighbourhood.txt")

In [83]:
%cat "neighbourhood.txt"

a: b(1)
b: a(1) c(1) d(1)
c: b(1) d(1)
d: b(1) c(1)
e:
f:


In [84]:
!wget https://github.com/pgordin/GraphsSN2025/raw/main/weighted0.txt

--2025-11-19 21:05:20--  https://github.com/pgordin/GraphsSN2025/raw/main/weighted0.txt
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pgordin/GraphsSN2025/main/weighted0.txt [following]
--2025-11-19 21:05:20--  https://raw.githubusercontent.com/pgordin/GraphsSN2025/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114 [text/plain]
Saving to: ‘weighted0.txt.2’

weighted0.txt.2     100%[===================>]     114  --.-KB/s    in 0s      

2025-11-19 21:05:21 (5.12 MB/s) - ‘weighted0.txt.2’ saved [114/114]



In [85]:
graph3 = Graph.from_edges("weighted0.txt")
print(graph3)

A: B(1) E(1)
B: A(1) C(1) D(1)
E: A(1) D(1) F(1) H(1)
C: B(1) D(1) F(1) G(1)
D: B(1) C(1) E(1) F(1)
F: C(1) D(1) E(1) G(1) H(1)
G: C(1) F(1) H(1) I(1)
H: E(1) F(1) G(1)
I: G(1)



In [86]:
# it works as well
!wget https://raw.githubusercontent.com/pgordin/GraphsSN2025/refs/heads/main/weighted0.txt

--2025-11-19 21:05:23--  https://raw.githubusercontent.com/pgordin/GraphsSN2025/refs/heads/main/weighted0.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114 [text/plain]
Saving to: ‘weighted0.txt.3’

weighted0.txt.3     100%[===================>]     114  --.-KB/s    in 0s      

2025-11-19 21:05:24 (2.46 MB/s) - ‘weighted0.txt.3’ saved [114/114]



In [87]:
graph4 = Graph.random_graph(10, 1/3)
print(graph4)

1: 6(1) 7(1) 8(1) 10(1)
2: 8(1) 9(1)
3: 4(1)
4: 3(1) 7(1) 10(1)
5: 6(1) 7(1) 8(1) 9(1)
6: 1(1) 5(1) 7(1) 8(1)
7: 1(1) 4(1) 5(1) 6(1)
8: 1(1) 2(1) 5(1) 6(1) 9(1)
9: 2(1) 5(1) 8(1)
10: 1(1) 4(1)



In [88]:
print(Graph.cycle(6))

1: 2(1) 6(1)
2: 1(1) 3(1)
3: 2(1) 4(1)
4: 3(1) 5(1)
5: 4(1) 6(1)
6: 5(1) 1(1)



### Part1

In [89]:
vertices = ["a", "b", "c", "d"]
matrix = np.array([[0,1,0,0],[1,0,1,0],[0,1,0,1],[0,0,0,1]])
print(vertices)
print(matrix)
print("---------------------------")
print_matrix(vertices, matrix)
print("---------------------------")
print_matrix(None,matrix)

['a', 'b', 'c', 'd']
[[0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]]
---------------------------
a :  b
b :  a  c
c :  b  d
d :  d
---------------------------
1 :  2
2 :  1  3
3 :  2  4
4 :  4


In [90]:
graph_dict = {
  "a": ["b"],
  "b": ["a", "c"],
  "c": ["b", "d"],
  "d": ["c"]
}
print(graph_dict)
print("---------------------------")
print_dict(graph_dict)

{'a': ['b'], 'b': ['a', 'c'], 'c': ['b', 'd'], 'd': ['c']}
---------------------------
a :  b
b :  a  c
c :  b  d
d :  c


In [91]:
graph1 = Graph.from_matrix(matrix, vertices)
print(graph1)

a: b
b: a c
c: b d
d: d



In [92]:
print(Graph(graph_dict))  # the same result

a: b
b: a c
c: b d
d: c



In [93]:
print(graph1.vertices())

['a', 'b', 'c', 'd']


In [94]:
print(graph1.matrix())

[[0 1 0 0]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]]


In [95]:
graph1.add_vertex("e")
print(graph1)

a: b
b: a c
c: b d
d: d
e:



In [96]:
graph1.add_edge(["e", "f"])
print(graph1)

a: b
b: a c
c: b d
d: d
e: f(1)
f: e(1)



In [97]:
graph1.add_arc(["e", "a"])  # breaking the symmetry
print(graph1)

a: b
b: a c
c: b d
d: d
e: f(1) a(1)
f: e(1)



In [98]:
graph1.add_edge(["e", "a"])  # restoring the symmetry
print(graph1)

a: b e
b: a c
c: b d
d: d
e: f(1) a(1)
f: e(1)



In [99]:
graph1.add_edge(["e", "f"]) # do nothing, an edge already exists
print(graph1)

a: b e
b: a c
c: b d
d: d
e: f(1) a(1)
f: e(1)



In [100]:
graph1.del_vertex("f")  # removing a vertex
print(graph1)

a: b e
b: a c
c: b d
d: d
e: a(1)



In [101]:
graph1.add_edge(["e", "e"]) # an error

ValueError: Loops are not allowed!

In [102]:
graph1.add_arc(["e", "e"]) # OK - loops are allowed in digraphs
print(graph1)

a: b e
b: a c
c: b d
d: d
e: a(1) e(1)



In [103]:
print(Graph.cycle(6))

1: 2(1) 6(1)
2: 1(1) 3(1)
3: 2(1) 4(1)
4: 3(1) 5(1)
5: 4(1) 6(1)
6: 5(1) 1(1)



## lab task 4

In [104]:
# =============================================================================
# TEST & DEMONSTRATION
# =============================================================================

if __name__ == "__main__":
    print("=== Task 1: Connected Components (BFS vs DFS) ===")
    # Create a disconnected graph
    g1 = Graph()
    g1.add_edge([1, 2])
    g1.add_edge([2, 3])
    g1.add_edge([4, 5])
    # Add a large linear component to test recursion depth if N was huge
    # But here just testing logic correctness

    print("Graph structure:\n", g1)

    print("DFS Recursive Components:")
    print(g1.connected_components())

    print("BFS Iterative Components:")
    print(g1.connected_components_bfs())
    print("\n")

    print("=== Task 2: Floyd-Warshall Efficiency Check ===")
    # Create a dense random graph for testing
    # Note: FW is O(V^3). Repeated BFS is O(V*(V+E)).
    # For dense graphs E approx V^2, so Repeated BFS is O(V^3) as well.
    # However, simple Python loops in FW are often slower than optimized collections
    # unless using vectorized numpy operations, but here we implement raw loops.

    N = 50
    P = 0.3
    print(f"Generating random graph G({N}, {P})...")
    g_perf = Graph.random_graph(N, P)

    # Measure Floyd-Warshall
    start_time = time.time()
    dist_matrix, _ = g_perf.floyd_warshall()
    fw_time = time.time() - start_time
    print(f"Floyd-Warshall time: {fw_time:.4f} sec")

    # Measure Repeated BFS (simulating distance calculation)
    # We use BFS from every node to find all-pairs shortest paths
    start_time = time.time()

    vertices = g_perf.vertices()
    all_bfs_dists = {}

    for start_node in vertices:
        # Simple BFS for distances
        distances = {v: float('inf') for v in vertices}
        distances[start_node] = 0
        queue = deque([start_node])
        while queue:
            u = queue.popleft()
            for v in g_perf.graph[u]:
                if distances[v] == float('inf'):
                    distances[v] = distances[u] + 1
                    queue.append(v)
        all_bfs_dists[start_node] = distances

    bfs_time = time.time() - start_time
    print(f"Repeated BFS time:   {bfs_time:.4f} sec")

    if fw_time < bfs_time:
        print("-> Floyd-Warshall was faster.")
    else:
        print("-> Repeated BFS was faster (common in sparse graphs or Python loop overhead).")

    print("\n")

    print("=== Task 3: Topological Sort ===")
    # Create a DAG
    dag = Graph()
    # 5 -> 0, 4 -> 0, 5 -> 2, 2 -> 3, 3 -> 1, 4 -> 1
    dag.add_arc([5, 0])
    dag.add_arc([4, 0])
    dag.add_arc([5, 2])
    dag.add_arc([2, 3])
    dag.add_arc([3, 1])
    dag.add_arc([4, 1])

    print("DAG Structure:\n", dag)

    try:
        sorted_order = dag.topological_sort()
        print("Topological Sort Order:", sorted_order)
    except ValueError as e:
        print(e)

    # Test Cycle Detection
    print("\nAdding cycle (1 -> 5)...")
    dag.add_arc([1, 5])
    try:
        dag.topological_sort()
    except ValueError as e:
        print("Expected Error caught:", e)

=== Task 1: Connected Components (BFS vs DFS) ===
Graph structure:
 1: 2(1)
2: 1(1) 3(1)
3: 2(1)
4: 5(1)
5: 4(1)

DFS Recursive Components:
[{1, 2, 3, 4, 5}, {1, 2, 3}, {4, 5}]
BFS Iterative Components:
[{1, 2, 3, 4, 5}, {1, 2, 3}, {4, 5}]


=== Task 2: Floyd-Warshall Efficiency Check ===
Generating random graph G(50, 0.3)...
Floyd-Warshall time: 0.1029 sec
Repeated BFS time:   0.0095 sec
-> Repeated BFS was faster (common in sparse graphs or Python loop overhead).


=== Task 3: Topological Sort ===
DAG Structure:
 5: 0(1) 2(1)
0:
4: 0(1) 1(1)
2: 3(1)
3: 1(1)
1:

Topological Sort Order: [5, 4, 2, 0, 3, 1]

Adding cycle (1 -> 5)...
Expected Error caught: Graph has a cycle! Topological sort not possible.


## Lab task 5

In [105]:
# =============================================================================
# TEST & DEMONSTRATION
# =============================================================================
if __name__ == "__main__":
    print("=== Task 1: Strongly Connected Components (Kosaraju) ===")
    # Creating a graph with known SCCs
    # 0->1, 1->2, 2->0 (SCC 1)
    # 1->3, 3->4 (Simple path)
    g_scc = Graph()
    g_scc.add_arc([0, 1])
    g_scc.add_arc([1, 2])
    g_scc.add_arc([2, 0])
    g_scc.add_arc([1, 3])
    g_scc.add_arc([3, 4])

    print("Graph for SCC:")
    print(g_scc)
    print("Strongly Connected Components:")
    sccs = g_scc.strongly_connected_components()
    for i, comp in enumerate(sccs):
        print(f"SCC {i+1}: {comp}")
    print("\n")

    print("=== Task 2: Weighted Matrix Conversion ===")
    gw = Graph()
    gw.add_weighted_arc(['A', 'B'], 5)
    gw.add_weighted_arc(['B', 'C'], 10)
    gw.add_weighted_arc(['A', 'C'], 2) # Shortcut

    print("Weighted Graph:")
    print(gw)

    mat, vmap = gw.to_weighted_matrix()
    print("Weighted Matrix (Rows/Cols: A, B, C):")
    print(mat)

    print("\nRecreating Graph from Matrix...")
    gw_recreated = Graph.from_weighted_matrix(mat, list(vmap.keys()))
    print(gw_recreated)
    print("\n")

    print("=== Task 3: Floyd-Warshall (Weighted) ===")
    print("Running FW on the weighted graph above...")
    dist_fw, vmap_fw = gw.floyd_warshall()
    print("Distance Matrix:")
    print(dist_fw)

    # Check: Dist(A->C) should be 2 (direct), Dist(A->B) is 5.
    # Dist(B->C) is 10.
    # What if we add a better path?
    # Let's add B->D->C with weights 1 and 1.
    gw.add_weighted_arc(['B', 'D'], 1)
    gw.add_weighted_arc(['D', 'C'], 1)

    print("Updated Graph with path B->D->C (1+1=2, better than B->C=10)")
    dist_fw2, _ = gw.floyd_warshall()

    a_idx = vmap_fw['A']
    b_idx = vmap_fw['B']
    c_idx = vmap_fw['C']

    print(f"Dist A->C (should be 2): {dist_fw2[a_idx][c_idx]}")
    print(f"Dist B->C (should be 2 via D, not 10): {dist_fw2[b_idx][c_idx]}")

=== Task 1: Strongly Connected Components (Kosaraju) ===
Graph for SCC:
0: 1(1)
1: 2(1) 3(1)
2: 0(1)
3: 4(1)
4:

Strongly Connected Components:
SCC 1: [0, 2, 1]
SCC 2: [3]
SCC 3: [4]


=== Task 2: Weighted Matrix Conversion ===
Weighted Graph:
A: B(5) C(2)
B: C(10)
C:

Weighted Matrix (Rows/Cols: A, B, C):
[[ 0.  5.  2.]
 [inf  0. 10.]
 [inf inf  0.]]

Recreating Graph from Matrix...
A: B(5.0) C(2.0)
B: C(10.0)
C:



=== Task 3: Floyd-Warshall (Weighted) ===
Running FW on the weighted graph above...
Distance Matrix:
[[ 0.  5.  2.]
 [inf  0. 10.]
 [inf inf  0.]]
Updated Graph with path B->D->C (1+1=2, better than B->C=10)
Dist A->C (should be 2): 2.0
Dist B->C (should be 2 via D, not 10): 2.0
